### Data sets were acquired from https://www.kaggle.com/c/titanic

In [1]:
# Import libraries and the data set
import pandas as pd
import numpy as np
import sidetable
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('train.csv', index_col=['PassengerId'])
X_test = pd.read_csv('test.csv', index_col=['PassengerId'])

In [2]:
# Have a look at the first five rows
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Column __Name__ and __Ticket__ has nothing to do with predicting __Survived__, it will be dropped

In [3]:
# Column 'Name' and 'Ticket' have nothing to do with predicting 'Survived', they will be dropped
df.drop(columns=['Name'], inplace=True)
X_test.drop(columns=['Name'], inplace=True)

In [4]:
# See the missing data in columns
df.stb.missing(style=True)

,missing,total,percent
Cabin,687,891,77.10%
Age,177,891,19.87%
Embarked,2,891,0.22%
Survived,0,891,0.00%
Pclass,0,891,0.00%
Sex,0,891,0.00%
SibSp,0,891,0.00%
Parch,0,891,0.00%
Ticket,0,891,0.00%
Fare,0,891,0.00%


As column __Cabin__ has 77% missing values, the column will be dropped.
<br>Column __Age__ can be imputed.
<br>As column __Embarked__ has 2 missing rows, the rows will be dropped.

In [5]:
# As Pclass includes fare, I will drop 'Fare'
df.drop(columns=['Ticket', 'Cabin'], inplace=True, axis=1)

In [6]:
X_test.stb.missing(style=True)

,missing,total,percent
Cabin,327,418,78.23%
Age,86,418,20.57%
Fare,1,418,0.24%
Pclass,0,418,0.00%
Sex,0,418,0.00%
SibSp,0,418,0.00%
Parch,0,418,0.00%
Ticket,0,418,0.00%
Embarked,0,418,0.00%


As column __Cabin__ has 77% missing values, the column will be dropped.
<br>Column __Age__ can be imputed.
<br>As column __Fare__ has 2 missing rows, the rows will be dropped.

In [7]:
# As column Cabin is missing a lot, and column Ticket has a lot of unique values, which is hard to encode, I will drop them
X_test.drop(columns=['Ticket', 'Cabin'], axis=1, inplace=True)

In [8]:
# Recheck the missing data in columns
df.stb.missing(style=True)

,missing,total,percent
Age,177,891,19.87%
Embarked,2,891,0.22%
Survived,0,891,0.00%
Pclass,0,891,0.00%
Sex,0,891,0.00%
SibSp,0,891,0.00%
Parch,0,891,0.00%
Fare,0,891,0.00%


In [9]:
X_test.stb.missing(style=True)

,missing,total,percent
Age,86,418,20.57%
Fare,1,418,0.24%
Pclass,0,418,0.00%
Sex,0,418,0.00%
SibSp,0,418,0.00%
Parch,0,418,0.00%
Embarked,0,418,0.00%


In [10]:
# Get X_train and y_train
y_train = df.Survived.copy()
X_train = df.drop(columns='Survived').copy()

In [11]:
# Retrive num_cols and cat_cols
num_cols = list(X_train._get_numeric_data().columns)
cat_cols = list(set(X_train.columns) - set(num_cols))

In [12]:
# Encode cat_cols
X_train = pd.get_dummies(data=X_train, columns=cat_cols, drop_first=True)
X_test = pd.get_dummies(data=X_test, columns=cat_cols, drop_first=True)

In [13]:
# Impute numeric data
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
num_imputer = IterativeImputer()
num_imputer.fit(X_train[num_cols])
X_train[num_cols] = num_imputer.transform(X_train[num_cols])
X_test[num_cols] = num_imputer.transform(X_test[num_cols])

In [14]:
def scale(X, num_cols):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X[num_cols])
    X_scaled = pd.DataFrame(data=X_scaled, columns=num_cols, index=X.index)
    return X_scaled

In [15]:
# Scaling data
X_scaled = scale(X_train, num_cols)
X_test_scaled = scale(X_test, num_cols)

In [16]:
# Update cat_cols
cat_cols = set(X_train.columns) - set(num_cols)

In [17]:
X_train = pd.concat([X_scaled, X_train[cat_cols]], axis=1)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=split_size, stratify=y_train)

In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

# Create the keras tuner model.
def build_model(hp):
    hp_drop_out = hp.Float('dropout', 0, 0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32), activation='relu'))
#         model.add(BatchNormalization())
        model.add(Dropout(hp_drop_out))
#     model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model

In [20]:
'''
Credit source: 
    https://www.tensorflow.org/tutorials/keras/keras_tuner
    https://github.com/keras-team/keras-tuner/blob/master/examples/cifar10.py
'''
import kerastuner as kt

tuner = kt.Hyperband(build_model,
                     objective='val_accuracy', 
                     max_epochs=16,
                     overwrite=False)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [21]:
es = EarlyStopping(monitor='val_accuracy', verbose=0, patience=10, min_delta=1e-3)
callbacks_list = [es]

In [22]:
tuner.search(X_train, y_train, epochs=64, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks_list)

INFO:tensorflow:Oracle triggered exit


In [23]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=64, validation_data=(X_val, y_val))

Epoch 1/64
20/20 [==============================] - 2s 43ms/step - loss: 0.6779 - accuracy: 0.5674 - val_loss: 0.5501 - val_accuracy: 0.6157
Epoch 2/64
20/20 [==============================] - 0s 15ms/step - loss: 0.5818 - accuracy: 0.6144 - val_loss: 0.4958 - val_accuracy: 0.8284
Epoch 3/64
20/20 [==============================] - 0s 15ms/step - loss: 0.5788 - accuracy: 0.7680 - val_loss: 0.4384 - val_accuracy: 0.8507
Epoch 4/64
20/20 [==============================] - 0s 15ms/step - loss: 0.4846 - accuracy: 0.8236 - val_loss: 0.4715 - val_accuracy: 0.8358
Epoch 5/64
20/20 [==============================] - 0s 14ms/step - loss: 0.4781 - accuracy: 0.8201 - val_loss: 0.4402 - val_accuracy: 0.8246
Epoch 6/64
20/20 [==============================] - 0s 15ms/step - loss: 0.4379 - accuracy: 0.8118 - val_loss: 0.3971 - val_accuracy: 0.8396
Epoch 7/64
20/20 [==============================] - 0s 15ms/step - loss: 0.4543 - accuracy: 0.8165 - val_loss: 0.3890 - val_accuracy: 0.8433
Epoch 8/64
20

In [24]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 40


In [25]:
# Re-instantiate the hypermodel and train it with the optimal number of epochs from above.
hypermodel = tuner.hypermodel.build(best_hps)
# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch)

Epoch 1/40
20/20 [==============================] - 2s 12ms/step - loss: 0.6677 - accuracy: 0.6068
Epoch 2/40
20/20 [==============================] - 0s 12ms/step - loss: 0.5707 - accuracy: 0.6879
Epoch 3/40
20/20 [==============================] - 0s 12ms/step - loss: 0.5058 - accuracy: 0.8231
Epoch 4/40
20/20 [==============================] - 0s 12ms/step - loss: 0.5082 - accuracy: 0.7938
Epoch 5/40
20/20 [==============================] - 0s 12ms/step - loss: 0.5034 - accuracy: 0.8197
Epoch 6/40
20/20 [==============================] - 0s 11ms/step - loss: 0.4563 - accuracy: 0.8082
Epoch 7/40
20/20 [==============================] - 0s 11ms/step - loss: 0.4494 - accuracy: 0.8217
Epoch 8/40
20/20 [==============================] - 0s 12ms/step - loss: 0.4015 - accuracy: 0.8340
Epoch 9/40
20/20 [==============================] - 0s 12ms/step - loss: 0.4132 - accuracy: 0.8282
Epoch 10/40
20/20 [==============================] - 0s 12ms/step - loss: 0.4416 - accuracy: 0.8198
Epoch 11/

In [26]:
eval_result = hypermodel.evaluate(X_val, y_val)
print("[test loss, test accuracy]:", eval_result)

9/9 [==============================] - 0s 3ms/step - loss: 0.7708 - accuracy: 0.8470
[test loss, test accuracy]: [0.7707520127296448, 0.8470149040222168]


In [27]:
# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch)

Epoch 1/40
20/20 [==============================] - 0s 11ms/step - loss: 0.3496 - accuracy: 0.8620
Epoch 2/40
20/20 [==============================] - 0s 11ms/step - loss: 0.3687 - accuracy: 0.8587
Epoch 3/40
20/20 [==============================] - 0s 12ms/step - loss: 0.3491 - accuracy: 0.8587
Epoch 4/40
20/20 [==============================] - 0s 13ms/step - loss: 0.3540 - accuracy: 0.8636
Epoch 5/40
20/20 [==============================] - 0s 11ms/step - loss: 0.3478 - accuracy: 0.8555
Epoch 6/40
20/20 [==============================] - 0s 13ms/step - loss: 0.3874 - accuracy: 0.8411
Epoch 7/40
20/20 [==============================] - 0s 12ms/step - loss: 0.3450 - accuracy: 0.8571
Epoch 8/40
20/20 [==============================] - 0s 12ms/step - loss: 0.3482 - accuracy: 0.8507
Epoch 9/40
20/20 [==============================] - 0s 13ms/step - loss: 0.3618 - accuracy: 0.8668
Epoch 10/40
20/20 [==============================] - 0s 12ms/step - loss: 0.3390 - accuracy: 0.8587
Epoch 11/

In [28]:
# from keras.models import save_model
# hypermodel.save('best_model.h5')

In [29]:
# Load the model and predict
# from keras.models import load_model
# hypermodel = load_model('best_model.h5')
y_pred = hypermodel.predict(X_test)
y_pred = y_pred.astype(int)

In [30]:
y_pred = pd.DataFrame(y_pred, columns=['Survived'], index=X_test.index)
# saving the dataframe 
y_pred.to_csv('Predictions.csv') 

In [31]:
import winsound
duration = 100  # milliseconds
freq = 3000  # Hz
winsound.Beep(freq, duration)